# Análisis de Frecuencias de trayectorias.

1. Creación del archivo con las trayectorias a identificar frecuencias `INFILE`
2. Ejecucion programa de creacion de frecuencias (`trajfreq`) para la creación del archivo con las frecuencias (`tfreq.bin`) 
   - Creacion archivo de etiquetas (`LABELS.CFG`)
   - Creacion archivo de Notas (`MAPTEXT.CFG`)
   - Visualización de las frecuencias

In [2]:
%%bash -l
#: Tilte      : crea_frec.sh
#: Date       : 2022-02-10
#: Author     : "Jose Agustin Garcia Reynoso <agustin@atmosfera.unam.mx>"
#: Version    : 1.0
#: Description: Realiza la secuencia de pasos para calcular la frecuencia
#:        de las trayectorias calculadas durante el periodo de 12 meses
# para cada mes
#: Options    : None
#
declare -a cmes=( ene ) #feb mar abr may jun jul ago sep 'oct' nov dic)
for i in ${!cmes[@]}
do
    # Se define en la variable DIR el nombre del directorio
    DIR=${cmes[10#$i]}
    
# se revisa si existe en el directorio $HOME si no se crea

if [ ! -d $HOME/data/$DIR/frecuencias ];then 
    mkdir $HOME/data/$DIR/frecuencias  ;fi

# se ubica en el directorio DIR
cd $HOME/data/$DIR/frecuencias

#se crea el archivo INFILE seleccionando las trayectorias

    ls  $HOME/data/$DIR/tdump*  > INFILE

# Directorio del hysplit
MDL="${HOME}/data/hysplit.v5.2.0_CentOS7.9.2009"

${MDL}/exec/trajfreq -ftfreq.bin -g0.1 -iINFILE -r1 -s0:5000 

#   -f[frequency file name (tfreq.bin)]
#   -g[grid size in degrees (1.0)]
#   -i[input file of file names (INFILE)]
#   -h[number of hours to include in analysis (9999)]
#   -r[residence time (0),1,2,3]:
#        (0) = no
#         1  = yes; divide endpoint counts by number of trajectories
#         2  = yes; divide endpoint counts by number of endpoints
#         3  = yes; divide endpoint counts by max count number for any grid cell
#   -c[include only endpoint files with same length as first endpoint file read]
#         (0) = no
#          1  = yes
#   -s[select bottom:top (0:99999) m AGL]

if [ -f LABELS.CFG.CFG ]; then rm LABELS.CFG ;fi

echo "'TITLE&','Frecuencia de trayectorias&'"  >LABELS.CFG
echo "'MAPID&',' Valores &'"             >>LABELS.CFG
echo "'UNITS&',' % &'"                   >>LABELS.CFG
echo "'VOLUM&',' &'"                    >>LABELS.CFG

cat >  MAPTEXT.CFG <<EOF
Instituto de Ciencias de la Atmosfera y Cambio Climatico
--------------------------------------------------
Hysplit Dispersion Calculation                   
--------------------------------------------------
Source Location: Laguna Verde                                  
Start Day:         01 ${DIR}                        
Start Time(UTC):       12                            
Meteorology Data Source:  ARW                        
Trajectory Computation Heights:    agl              
Pollutant Emission Rate:                          
Initial Pollutant Distribution:   2                
Deposition Options Enabled:        No               
Notes:                                            
                                                  
Issued: $(date +%c)                                           
--------------------------------------------------
EOF
echo "'$MDL/graphics/mexico.shp' 0 0.01 0.4 0.6 0.8" >shapefiles_mex.txt
echo "'$MDL/graphics/carre1mgw.shp' 0 0.005 0.8 0.7 0.5" >>shapefiles_mex.txt


${MDL}/exec/concplot +g0 -m0 -k1 -z80 -jshapefiles_mex.txt -itfreq.bin
convert concplot.ps ${DIR}_frec.jpg
mv ${DIR}_frec.jpg ../../
#   -a[Arcview GIS: 0-none 1-log10 2-value 3-KML 4-partial KML]
#   +a[KML altitude mode: (0)-clampedToGround, 1-relativeToGround]
#   -A[KML options: 0-none 1-KML with no extra overlays]
#   -b[Bottom display level: (0) m]
#   -c[Contours: (0)-dyn/exp 1-fix/exp 2-dyn/lin 3-fix/lin 4-set 50-0,interval 10 51-1,interval 10]
#   +c[Write contour values to text output file CONTUR: (0)-no 1-yes]
#   -d[Display: (1)-by level, 2-levels averaged]
#   -e[Exposure units flag: (0)-concentrations, 1-exposure, 2-chemical threshold,
#                                 3-hypothetical volcanic ash, 4-mass loading]
#   -f[Frames: (0)-all frames one file, 1-one frame per file]
#   -g[Circle overlay: ( )-auto, #circ(4), #circ:dist_km]
#   +g[Graphics type: (0)-Postscript, 1-SVG]
#   -h[Hold map at center lat-lon: (source point), lat:lon]
#   -i[Input file name: (cdump)]
#   -j[Graphics map background file name: (arlmap) or shapefiles.<(txt)|process suffix>]]
#   -k[Kolor: 0-B&W, (1)-Color, 2-No Lines Color, 3-No Lines B&W]
#   -l[Label options: ascii code, (73)-open star]
#   +l[Use THIS IS A TEST label: (0)-no, 1-yes]
#   -L[LatLonLabels: none=0 auto=(1) set=2:value(tenths)]
#   -m[Map projection: (0)-Auto 1-Polar 2-Lamb 3-Merc 4-CylEqu]
#   +m[Maximum square value: 0=none, (1)=both, 2=value, 3=square]
#   -n[Number of time periods: (0)-all, numb, min:max, -incr]
#   -o[Output file name: (concplot.ps)]
#   -p[Process file name suffix: (ps) or process ID]
#   -x[Concentration multiplier: (1.0)]
#   -y[Deposition multiplier:    (1.0)]
#   -z[Zoom factor: 0-least zoom, (50), 100-most zoom]
echo "### Se termió de hacer el mes de $DIR ###"
echo  --------------------------------------------------
done
unset cmes

 Input files processed:          124
 Count sum over entire domain:    1588.00000    
 Total number of traj:          124
 Total endpoint count:         1464
 Endpoints/Trajectory:           11
 Complete Trajfreq
 Started Concentration Drawing
 HYSPLIT version: hysplit.v5.2.0
 Last Changed Date: 2022-01-16
 Finished map:            1
 Complete Concplot:           1  time periods
                                                                                 
### Se termió de hacer el mes de ene ###
--------------------------------------------------
